# Experiment #38 - Represent Learn Content By Wiki Articles
We aim here to represente each learn content (and its subdivisions) as a set of wikipedia articles in order to have "blocks" that the learn content would be made from. We may use articles links as priors to help create these representations.

In [19]:
from wikipydia import wikipedia, url

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.util import ngrams

from string import punctuation

from learndata import LearnContents

#Load learn contents
ld = LearnContents()

stopwords = list(punctuation)

In [38]:
tokens = word_tokenize(ld[0].title)
list(ngrams(tokens, 0))

[('Data',), ('Visualization',), ('with',), ('d3.js',)]

In [119]:
def clear_bad_chars(text):
    """Ensure bad characters (such as '–' that is not a common '-') be removed."""
    return text.encode("ascii", "ignore").decode("ascii")

In [120]:
def get_ngrams(text, max_ngram_size=-1):
    """Tokenize text and them return its ngrams."""
    
    tokens = [word for word in word_tokenize(clear_bad_chars(text.strip())) if word not in stopwords]
    
    if max_ngram_size == -1:
        max_ngram_size = len(tokens)
    
    ngrams_list = list()
    for ngram_size in range(1, max_ngram_size+1):
        for ngram_tuple in ngrams(tokens, ngram_size):
            ngram_text = " ".join(ngram_tuple)
            ngrams_list.append(ngram_text)

    return ngrams_list

In [121]:
def print_lc_content(lc):
    print(lc)
    for lc in lc:
        print("\t", lc)
        for lc in lc:
            print("\t\t", lc)

In [122]:
top_contents = [lc for lc in ld if lc.parent == None]

In [157]:
maybe use markov models to find the best links

SyntaxError: invalid syntax (<ipython-input-157-336adb72b465>, line 1)

In [156]:
print_lc_content(top_contents[303])

Implementing Cloud Storage with OpenStack Swift
	 Chapter 1: Cloud Storage: Why Can't I be like Google?
		 Elements of cloud storage
		 Object storage
		 OpenStack Swift
		 Summary
	 Chapter 2: OpenStack Swift Architecture
		 The logical organization of objects
		 The Swift implementation
		 Additional features
		 Summary
	 Chapter 3: Installing OpenStack Swift
		 Hardware planning
		 Server setup and network configuration
		 Summary
	 Chapter 4: Using Swift
		 Installing the clients
		 Creating a token using authentication
		 Displaying metadata information for an account, container, or object
		 Listing containers
		 Listing objects in a container
		 Updating the metadata for a container
		 Environment variables
		 Pseudo-hierarchical directories
		 Container ACLs
		 Transferring large objects
		 Amazon S3 API compatibility
		 Accessing Swift using client libraries
		 Summary
	 Chapter 5: Managing Swift
		 Routine management
		 Logging using rsyslog
		 Failure management
		 Capacity 

In [158]:
for ngram_text in get_ngrams(top_contents[303].title):
    try:
        art = wikipedia.get_article_by_title(url.UnquotedURL(ngram_text))
        print(ngram_text, " -> ", art.title())
    except:
        #continue
        print("Not found: ", ngram_text)

Not found:  Implementing
Cloud  ->  Cloud
Storage  ->  Storage
with  ->  With
OpenStack  ->  OpenStack
Swift  ->  Swift
Not found:  Implementing Cloud
Cloud Storage  ->  Cloud storage
Not found:  Storage with
Not found:  with OpenStack
OpenStack Swift  ->  OpenStack
Not found:  Implementing Cloud Storage
Not found:  Cloud Storage with
Not found:  Storage with OpenStack
Not found:  with OpenStack Swift
Not found:  Implementing Cloud Storage with
Not found:  Cloud Storage with OpenStack
Not found:  Storage with OpenStack Swift
Not found:  Implementing Cloud Storage with OpenStack
Not found:  Cloud Storage with OpenStack Swift
Not found:  Implementing Cloud Storage with OpenStack Swift


In [67]:
art = wikipedia.get_article_by_title(url.UnquotedURL("Table"))

In [69]:
art.links()

[['Table_(furniture)', 'Table (furniture)'],
 ['Table_(information)', 'Table (information)'],
 ['Table_(database)', 'Table (database)'],
 ['Calligra_Tables', 'Calligra Tables'],
 ['Mathematical_table', 'Mathematical table'],
 ['Table_(landform)', 'Table (landform)'],
 ['Table_(parliamentary_procedure)', 'Table (parliamentary procedure)'],
 ['Tables_(board_game)', 'Tables (board game)'],
 ['The_Table', 'The Table'],
 ['Sound_board_(music)', 'sound board (music)'],
 ['Al-Ma%27ida', "Al-Ma'ida"],
 ['Water_table', 'Water table']]